In [9]:
import os
import sys
import pandas as pd
import xarray as xr
import numpy as np
from numpy import ones
from numpy.linalg import cholesky
from pandas_plink import read_plink1_bin
from limix.qc import quantile_gaussianize
from sklearn.preprocessing import StandardScaler 
from cellregmap import run_interaction, estimate_betas

In [2]:
interaction_eqtl_file = "/project2/gilad/jpopp/ebQTL/results/cellregmap_eqtl_calling/eb_cellid/pseudobulk_tmm/basic/all_genes_merged.mash-signif.fasttopics_10topics.cellregmap.sighits.tsv"
sample_mapping_file = "/project2/gilad/jpopp/ebQTL/data/cellregmap/pseudocell_metadata.tsv"
genotype_file = "/project2/gilad/jpopp/ebQTL/data/genotypes/yri_maf0.1_all.hg38.bed"
kinship_file = "/project2/gilad/jpopp/ebQTL/data/genotypes/yri_kinship.tsv"
phenotype_file = "/project2/gilad/jpopp/ebQTL/data/single_cell_objects/eb_pseudocells_normalized.nc"
cell_context_file = "/project2/gilad/jpopp/ebQTL/results/fast_topics/fasttopics_10topics_loadings.tsv"

In [28]:
g = "TMEM213"

In [29]:
# ############################################
# ############### Gene file ##################
# ############################################


######################################
#####Filter on specific gene-SNP pairs
interaction_eqtls = pd.read_csv(interaction_eqtl_file, sep="\t")

############################################
########## Sample mapping file #############
############################################

## this file will map pseudocells to donors 
## it will also only include donors we have single-cell data for
sample_mapping = pd.read_csv(sample_mapping_file, sep="\t").sort_values(by=['donor_id', 'pseudocell'])

## donor_id are donor IDs, as found in the genotype matrix (G) and GRM covariance (K)
## cell are cell IDs, as found in the scRNA-seq phenotype vector (y) and cell context covariance (C)

## extract unique individuals
donors = sample_mapping["donor_id"].unique()
print("Number of unique donors: {}".format(len(donors)))

############################################
############# Kinship matrix ###############
############################################

## read in GRM (genotype relationship matrix; kinship matrix)
#kinship_file=input_files_dir+"kinship_file.csv"
K = pd.read_csv(kinship_file, sep = "\t", index_col = 0)
assert all(K.columns == K.index) #symmetric matrix, donors x donors

# Make the dataframe as DataArray
K = xr.DataArray(K.values, dims=["donor_0", "donor_1"], coords={"donor_0": K.columns, "donor_1": K.index})
K = K.sortby("donor_0").sortby("donor_1")

# Find the shared donors between kinship and expression data
donors = sorted(set(list(K.donor_0.values)).intersection(donors))
print("Number of donors after kinship intersection: {}".format(len(donors)))

# subset to relevant donors
K = K.sel(donor_0=donors, donor_1=donors)
assert all(K.donor_0 == donors)
assert all(K.donor_1 == donors)

# Decompose such as K = hK @ hK.T
hK = cholesky(K.values)
hK = xr.DataArray(hK, dims=["sample", "col"], coords={"sample": K.donor_0.values})
assert all(hK.sample.values == K.donor_0.values)

del K

############################################
##### expand from donors to cells ##########

# Expand hK from donors to cells
hK_expanded = hK.sel(sample=sample_mapping["donor_id"].values)
assert all(hK_expanded.sample.values == sample_mapping["donor_id"].values)


#####################################
############ Phenotypes #############
#####################################

# Phenotype (pseudocell expression)
phenotype = xr.open_dataarray(phenotype_file, autoclose=True)
assert all(phenotype.pseudo_cell.values == sample_mapping["pseudocell"].values)
######################################
########## Cell contexts #############
######################################

# cellular environments
C = pd.read_csv(cell_context_file, index_col = 0, sep = "\t")

# filter to the more interpretable topics
C = C[['k1','k2', 'k4','k5','k6', 'k7', 'k8', 'k10']]

C = xr.DataArray(C.values, dims=["pseudocell", "topic"], coords={"pseudocell": C.index.values, "topic": C.columns.values})
assert all(C.pseudocell.values == sample_mapping["pseudocell"].values)

# normalize cell contexts
scaler = StandardScaler()
scaler.fit(C.values)
C.values = scaler.transform(C.values)

######################################
############ Covariates ##############
######################################

cov = sample_mapping.drop(columns='donor_id').set_index('pseudocell')
cov['intercept'] = 1

#####################################
############ Genotypes ##############
#####################################

## read in genotype file (plink format)
#plink_file = snakemake_input[1]
G = read_plink1_bin(genotype_file)
# Select snps appearing for that gene
qtl_snp = interaction_eqtls[interaction_eqtls['EB_HGNC']==g]['EB_VARIANT_ID'].unique()

Number of unique donors: 51
Number of donors after kinship intersection: 51


Mapping files: 100%|██████████| 3/3 [00:06<00:00,  2.10s/it]


In [30]:
qtl_snp

array(['rs6467807'], dtype=object)

In [27]:
qtl_snp = ['rs10100742', 'rs4510815']
G1 = G[:,G['snp'].isin(qtl_snp)].sel(sample=sample_mapping["donor_id"].values)
print(G1.snp.values)

qtl_snp2 = ['rs4510815', 'rs10100742']
G2 = G[:,G['snp'].isin(qtl_snp2)].sel(sample=sample_mapping["donor_id"].values)
print(G2.snp.values)

['rs4510815' 'rs10100742']
['rs4510815' 'rs10100742']


In [4]:
G_expanded = G[:,G['snp'].isin(qtl_snp)].sel(sample=sample_mapping["donor_id"].values)
assert all(hK_expanded.sample.values == G_expanded.sample.values)

print("G_tested shape is {}".format(G_expanded.shape))

######################################
############# Run CRM ################
######################################
#For each gene on specific chromosome

""" Input list: 
    y: n x 1 (only one gene tested at a time)
    W: n x c, where c is the number of fixed effect covariates (e.g., age, sex..)
    C: n x k, where k is the number of contexts to test for interactions
    G: n x s, where s is the number of SNPs to be tested for a given gene
    hK: n x p, where p is the number of individuals, decomposition of the n x n kinship matrix K
"""

# expression for each gene
y = phenotype.sel(gene=g)
y = quantile_gaussianize(y)
y = y.values.reshape(y.shape[0],1)

W = cov.values
C_val = C.values
G_val = G_expanded.values
hK_val = hK_expanded.values

print("Running for gene {}".format(g))

# run interaction test using CellRegMap
betas = estimate_betas(y=y, W=W, G=G_val, E=C_val, hK=hK_val)

beta_gxc_df = pd.DataFrame({'PSEUDOCELL': phenotype.pseudo_cell.values,
                            'BETA_GXC': betas[1][0, :, 0]})
beta_gxc_df['EB_HGNC'] = g
beta_gxc_df['EB_VARIANT_ID'] = qtl_snp[0]
beta_gxc_df['BETA_G'] = betas[0][0]

Number of unique donors: 51
Number of donors after kinship intersection: 51


Mapping files: 100%|██████████| 3/3 [00:06<00:00,  2.08s/it]


G_tested shape is (17913, 1)
Running for gene GATA4
[0.44598895]


In [13]:
pvals = run_interaction(y=y, W=W, G=G_val, E=C_val, hK=hK_val)
pvals

100%|██████████| 1/1 [01:59<00:00, 119.96s/it]


(array([0.41967181]),
 {'rho1': array([0.8]),
  'e2': array([0.0267657]),
  'g2': array([0.00669143]),
  'eps2': array([0.32264962])})

In [15]:
qtl_snp

array(['rs4510815'], dtype=object)

In [19]:
G_alt = G[:,G['snp'].isin(['rs10100742'])].sel(sample=sample_mapping["donor_id"].values)

In [20]:
G_alt

<xarray.DataArray 'genotype' (sample: 17913, variant: 1)>
dask.array<getitem, shape=(17913, 1), dtype=float32, chunksize=(17913, 1), chunktype=numpy.ndarray>
Coordinates: (12/14)
  * sample   (sample) object 'NA18486' 'NA18486' ... 'NA19257' 'NA19257'
  * variant  (variant) <U14 'variant1732982'
    fid      (sample) object 'NA18486' 'NA18486' ... 'NA19257' 'NA19257'
    iid      (sample) object 'NA18486' 'NA18486' ... 'NA19257' 'NA19257'
    father   (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    mother   (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    ...       ...
    chrom    (variant) object 'chr8'
    snp      (variant) object 'rs10100742'
    cm       (variant) float64 0.0
    pos      (variant) int32 11688938
    a0       (variant) object 'C'
    a1       (variant) object 'T'

In [21]:
G_expanded

<xarray.DataArray 'genotype' (sample: 17913, variant: 1)>
dask.array<getitem, shape=(17913, 1), dtype=float32, chunksize=(17913, 1), chunktype=numpy.ndarray>
Coordinates: (12/14)
  * sample   (sample) object 'NA18486' 'NA18486' ... 'NA19257' 'NA19257'
  * variant  (variant) <U14 'variant1732972'
    fid      (sample) object 'NA18486' 'NA18486' ... 'NA19257' 'NA19257'
    iid      (sample) object 'NA18486' 'NA18486' ... 'NA19257' 'NA19257'
    father   (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    mother   (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    ...       ...
    chrom    (variant) object 'chr8'
    snp      (variant) object 'rs4510815'
    cm       (variant) float64 0.0
    pos      (variant) int32 11675875
    a0       (variant) object 'G'
    a1       (variant) object 'A'

In [22]:
pvals2 = run_interaction(y=y, W=W, G=G_alt.values, E=C_val, hK=hK_val)
pvals2

100%|██████████| 1/1 [01:59<00:00, 119.11s/it]


(array([0.00039561]),
 {'rho1': array([0.8]),
  'e2': array([0.02685194]),
  'g2': array([0.00671299]),
  'eps2': array([0.32255157])})

In [24]:
G_alt.snp.values

array(['rs10100742'], dtype=object)

In [26]:
pd.DataFrame({"EB_VARIANT_ID":G_alt.snp.values})

,EB_VARIANT_ID
0,rs10100742
